<a href="https://colab.research.google.com/github/krystal826/Natural-Language-Processing/blob/main/Lab09_Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
class Dictlist(dict):
    
    def __setitem__(self, key, value):
        try:
            self[key]
        except KeyError:
            super(Dictlist, self).__setitem__(key, [])
        self[key].append(value)


class production_rule(object):
    
    result = None
    p1 = None
    p2 = None
    
    #Parameters:
    #   Result: String
    #   p1: Production rule (left child of the production rule)
    #   p2: Production rule (right child of the production rule)
    def __init__(self,result,p1,p2):
        self.result = result
        self.p1 = p1
        self.p2 = p2
    
    #Returns the result of the production rule, VP, S, NP... 
    @property
    def get_type(self):
        return self.result
    
    #Returns the left child of the production rule
    @property
    def get_left(self):
        return self.p1
    
    #Returns the right child of the production rule
    @property
    def get_right(self):
        return self.p2

class Cell(object):
    productions = []
    
    
    #Parameters:
    #   Productions: List of production rules
    
    def __init__(self, productions=None):
        if productions is None:
            self.productions = []
        else:
            self.productions = productions
            
    def add_production(self, result,p1,p2):
        self.productions.append(production_rule(result,p1,p2))
    
    def set_productions(self, p):
        self.productions = p
    
    @property
    def get_types(self):
        types = []
        for p in self.productions:
            types.append(p.result)
        return types
    @property
    def get_rules(self):       
        return self.productions


class Grammar(object):
    
    grammar_rules = Dictlist()
    parse_table = None
    length = 0
    tokens = []
    number_of_trees = 0
    
    #Parameters:
    #   Filename: file containing a grammar
    
    def __init__(self, filename):
        self.grammar_rules = Dictlist()
        self.parse_table = None
        self.length = 0
        for line in open(filename):
            a, b = line.split("->")
            self.grammar_rules[b.rstrip().strip()]=a.rstrip().strip()
        
        if len(self.grammar_rules) == 0:
            raise ValueError("No rules found in the grammar file")
        print('')
        print('Grammar file readed succesfully. Rules readed:')
        self.print_rules()
        print('')
    
    #Print the production rules in the grammar
    
    def print_rules(self):
        for r in self.grammar_rules:
            for p in self.grammar_rules[r]:
                print(str(p) + ' --> ' + str(r))
        
    def apply_rules(self,t):
        try:
            return self.grammar_rules[t]
        except KeyError as r:
            return None
            
    #Parse a sentence (string) with the CYK algorithm   
    def parse(self,sentence):
        self.number_of_trees = 0
        self.tokens = sentence.split()
        self.length = len(self.tokens)
        if self.length < 1:
            raise ValueError("The sentence could no be read")
        self.parse_table = [ [Cell() for x in range(self.length - y)] for y in range(self.length) ]
        
         #Process the first line
        
        for x, t in enumerate(self.tokens):
            
            r = self.apply_rules(t)
            if r == None:
                raise ValueError("The word " + str(t) + " is not in the grammar")
            else:
                for w in r: 
                    self.parse_table[0][x].add_production(w,production_rule(t,None,None),None)
        
        #Run CYK-Parser      
        for l in range(2,self.length+1):
            for s in range(1,self.length-l+2):
                for p in range(1,l-1+1):
                    
                    t1 = self.parse_table[p-1][s-1].get_rules
                    t2 = self.parse_table[l-p-1][s+p-1].get_rules
                            
                    for a in t1:
                        for b in t2:
                            r = self.apply_rules(str(a.get_type) + " " + str(b.get_type))
                                    
                            if r is not None:
                                for w in r:
                                    print('Applied Rule: ' + str(w) + '[' + str(l) + ',' + str(s) + ']' + ' --> ' + str(a.get_type) + '[' + str(p) + ',' + str(s) + ']' + ' ' + str(b.get_type)+ '[' + str(l-p) + ',' + str(s+p) + ']')
                                    self.parse_table[l-1][s-1].add_production(w,a,b)
                               
        self.number_of_trees = len(self.parse_table[self.length-1][0].get_types)
        if  self.number_of_trees > 0:
            print("----------------------------------------")
            print('The sentence IS accepted in the language')
            print('Number of possible trees: ' + str(self.number_of_trees))
            print("----------------------------------------")
        else:
            print("--------------------------------------------")
            print('The sentence IS NOT accepted in the language')
            print("--------------------------------------------")
        
        
    #Returns a list containing the parent of the possible trees that we can generate for the last sentence that have been parsed
    def get_trees(self):
        return self.parse_table[self.length-1][0].productions
                
                
    #@TODO
    def print_trees(self):
        pass
                      
    #Print the CYK parse trable for the last sentence that have been parsed.             
    def print_parse_table(self):
        try:
            from tabulate import tabulate
        except (ModuleNotFoundError,ImportError) as r:
            import subprocess
            import sys
            import logging
            logging.warning('To print the CYK parser table the Tabulate module is necessary, trying to install it...')
            subprocess.call([sys.executable, "-m", "pip", "install", 'tabulate'])

            try:
                from tabulate import tabulate
                logging.warning('The tabulate module has been instaled succesfuly!')

            except (ModuleNotFoundError,ImportError) as r:
                logging.warning('Unable to install the tabulate module, please run the command \'pip install tabulate\' in a command line')

        
        lines = [] 
        
        for row in reversed(self.parse_table):
            l = []
            for cell in row:
                l.append(cell.get_types)
            lines.append(l)
        
        lines.append(self.tokens)
        print('')
        print(tabulate(lines))
        print('')

In [12]:
g = Grammar('grammar lab9 task2.txt')


Grammar file readed succesfully. Rules readed:
S --> NP VP
S --> VP
S --> Aux NP AdjP
S --> WH-NP Aux NP
NP --> Det N
NP --> Poss N
NP --> Pronoun
VP --> V NP
VP --> V Adj
VP --> V
AdjP --> Adj
Adj --> quiet
Adj --> hungry
Aux --> are
Aux --> is
Det --> a
N --> dog
N --> cat
N --> name
Poss --> your
Pronoun --> you
Pronoun --> I
Pronoun --> they
V --> chased
V --> keep
WH-NP --> what
WH-NP --> where



In [13]:
g.parse('a dog chased a cat')

Applied Rule: NP[2,1] --> Det[1,1] N[1,2]
Applied Rule: NP[2,4] --> Det[1,4] N[1,5]
Applied Rule: VP[3,3] --> V[1,3] NP[2,4]
Applied Rule: S[5,1] --> NP[2,1] VP[3,3]
----------------------------------------
The sentence IS accepted in the language
Number of possible trees: 1
----------------------------------------


In [14]:
#Print the CYK parse table for the last sentence that have been parsed.
print("Parse Table")
g.print_parse_table()

Parse Table

-------  -----  ------  -------  -----
['S']
[]       []
[]       []     ['VP']
['NP']   []     []      ['NP']
['Det']  ['N']  ['V']   ['Det']  ['N']
a        dog    chased  a        cat
-------  -----  ------  -------  -----



In [15]:
g.parse('keep quiet')
g.print_parse_table()

Applied Rule: VP[2,1] --> V[1,1] Adj[1,2]
----------------------------------------
The sentence IS accepted in the language
Number of possible trees: 1
----------------------------------------

------  -------
['VP']
['V']   ['Adj']
keep    quiet
------  -------



In [16]:
g.parse('are you hungry')
g.print_parse_table()

--------------------------------------------
The sentence IS NOT accepted in the language
--------------------------------------------

-------  -----------  -------
[]
[]       []
['Aux']  ['Pronoun']  ['Adj']
are      you          hungry
-------  -----------  -------



In [17]:
g.parse('where are you')
g.print_parse_table()

--------------------------------------------
The sentence IS NOT accepted in the language
--------------------------------------------

---------  -------  -----------
[]
[]         []
['WH-NP']  ['Aux']  ['Pronoun']
where      are      you
---------  -------  -----------

